In [148]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error
from sklearn.dummy import DummyRegressor

import xgboost as xgb
from catboost import CatBoostRegressor as ctb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold
import eli5
import re
from sklearn.metrics import mean_absolute_error

# Wczytywanie danych

In [94]:
df_train = pd.read_hdf("../input/df.train.h5")
df_test = pd.read_hdf("../input/df.test.h5")

print(df_train.shape)

df_train.sample(10)

(80129, 10)


,id,price_details,price_value,price_currency,features,seller_name,seller_type,seller_address,breadcrumb,offer_params
94186,94186,Cena Brutto,5 999,PLN,"[ABS, Elektryczne szyby przednie, Poduszka pow...",None,Osoba prywatna,"Kiełczów, wrocławski, Dolnośląskie","[Osobowe, Dacia, Sandero, Dacia Sandero I (200...","{'Pojemność skokowa': '1 299 cm3', 'Model poja..."
65077,65077,"Cena Brutto, Do negocjacji",25 900,PLN,"[ABS, Elektryczne szyby przednie, Poduszka pow...","62-020 Swarzędz-Jasin, Ul. Sadowa 20.",Dealer,"Jasin, sadowa 20 - 62-020 Swarzędz, poznański,...","[Osobowe, Audi, A6, Audi A6 C6 (2004-2011)]","{'Pojemność skokowa': '2 000 cm3', 'Model poja..."
109534,109534,Cena Brutto,27 000,PLN,[],None,Osoba prywatna,"Ostrowiec, strzelecko-drezdenecki, Lubuskie","[Osobowe, BMW, X5, BMW X5 E53 (1999-2006)]","{'Pojemność skokowa': '3 000 cm3', 'Model poja..."
55558,55558,"Cena Brutto, Do negocjacji",15 300,PLN,"[ABS, Immobilizer, Radio fabryczne, Centralny ...",None,Osoba prywatna,"Tarnowo Podgórne, poznański, Wielkopolskie","[Osobowe, Škoda, Roomster]","{'Kolor': 'Czarny', 'Model pojazdu': 'Roomster..."
103183,103183,"Cena Brutto, Faktura VAT",8 900,PLN,"[ABS, Elektryczne szyby przednie, Poduszka pow...",None,Osoba prywatna,"Poznań, Wielkopolskie, Grunwald","[Osobowe, Renault, Scenic, Renault Scenic II (...","{'Pojemność skokowa': '1 461 cm3', 'Model poja..."
31233,31233,Cena Brutto,28 500,PLN,"[ABS, Elektryczne szyby przednie, Poduszka pow...",None,Osoba prywatna,"Łowicz, łowicki, Łódzkie","[Osobowe, Opel, Astra, Opel Astra J (2009-2015)]","{'Bezwypadkowy': 'Tak', 'Liczba miejsc': '5', ..."
70429,70429,Cena Brutto,71 900,PLN,"[ABS, Alarm, Czujniki parkowania przednie, Ele...",AUTO KOMIS FOKS,Dealer,"Stawowa 7a - 43-230 Goczałkowice-Zdrój, pszczy...","[Osobowe, Škoda, Superb, Škoda Superb II (2008-)]","{'Bezwypadkowy': 'Tak', 'Liczba miejsc': '5', ..."
4147,4147,"Cena Brutto, Do negocjacji",25 000,PLN,"[ABS, Elektryczne szyby przednie, Poduszka pow...",F.H.U. VERSUS S.C. Dealer Samochodów Używanych...,Dealer,"ul. Morska 212 A - 81-006 Gdynia, Chylonia (Po...","[Osobowe, Audi, A6, Audi A6 C6 (2004-2011)]","{'Bezwypadkowy': 'Tak', 'Liczba miejsc': '5', ..."
67609,67609,Cena Brutto,14 200,PLN,"[ABS, Elektryczne szyby przednie, Poduszka pow...",None,Osoba prywatna,"Ostrowiec Świętokrzyski, ostrowiecki, Świętokr...","[Osobowe, Opel, Zafira, Opel Zafira B (2005-20...","{'Pojemność skokowa': '1 900 cm3', 'Model poja..."
18025,18025,"Cena Brutto, Do negocjacji, Faktura VAT",41 900,PLN,"[ABS, Elektryczne szyby przednie, Poduszka pow...",Salon samochodowy SilesiaMoto,Dealer,"Chorzowska 52 - 40-001 Katowice, Śląskie (Polska)","[Osobowe, Peugeot, 508]","{'Kolor': 'Czarny', 'Model pojazdu': '508', 'L..."


In [95]:
#price remove whitespaces; replace , to .
df_train["price_value"].map(lambda x: x.replace(" ", "").replace(",", ".")).astype(float)
df_train["price_value"] = df_train["price_value"].str.replace(" ", "").str.replace(",", ".").astype(float)

In [96]:
df = pd.concat([df_train, df_test])

In [97]:
def check_missing():
    for column in df.columns:
        missing = column, df[column].isnull().sum()
        if missing[1] == 0: continue
        print(missing)

check_missing()

('price_details', 80129)
('price_value', 80129)
('price_currency', 80129)
('seller_name', 63159)


In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160258 entries, 7 to 160254
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              160258 non-null  int64  
 1   price_details   80129 non-null   object 
 2   price_value     80129 non-null   float64
 3   price_currency  80129 non-null   object 
 4   features        160258 non-null  object 
 5   seller_name     97099 non-null   object 
 6   seller_type     160258 non-null  object 
 7   seller_address  160258 non-null  object 
 8   breadcrumb      160258 non-null  object 
 9   offer_params    160258 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 13.4+ MB


# Feature engineering

In [99]:
# offer_params to columns
params = df["offer_params"].apply(pd.Series)
params

,Bezwypadkowy,Liczba miejsc,Kraj pochodzenia,Pojemność skokowa,Rodzaj paliwa,Skrzynia biegów,Możliwość finansowania,Pierwszy właściciel,Oferta od,Liczba drzwi,...,First registration,Pearl,Zarejestrowany jako zabytek,VAT free,Engine Code,Particle filter,Leasing concession,Damaged,CO2 emissions,Monthly payment value
7,Tak,5,Polska,1 199 cm3,Benzyna,Manualna,Tak,Tak,Firmy,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,5,Polska,1 598 cm3,Benzyna,Automatyczna bezstopniowa (CVT),Tak,Tak,Firmy,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Tak,5,Niemcy,1 364 cm3,Benzyna,Manualna,NaN,Tak,Firmy,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,5,Niemcy,1 798 cm3,Benzyna,Manualna,NaN,NaN,Osoby prywatnej,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,Tak,5,Niemcy,1 250 cm3,Benzyna,Manualna,NaN,NaN,Osoby prywatnej,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160245,NaN,NaN,NaN,2 000 cm3,Diesel,"Automatyczna dwusprzęgłowa (DCT, DSG)",NaN,NaN,Osoby prywatnej,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160246,NaN,4,Dania,998 cm3,Benzyna,Manualna,NaN,Tak,Firmy,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160247,Tak,5,Niemcy,1 600 cm3,Benzyna,Manualna,Tak,Tak,Firmy,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160251,NaN,5,Francja,1 560 cm3,Diesel,Manualna,NaN,NaN,Osoby prywatnej,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
#concatenate and remove duplicated columns
if 'Door count' in params:
    params['Liczba drzwi'] = params['Liczba drzwi'].fillna(params['Door count'])
    del params['Door count']
if 'VAT discount' in params:
    params['VAT free'] = params['VAT free'].fillna(params['VAT discount'])
    del params['VAT discount']
if 'Service record' in params:
    params['Serwisowany w ASO'] = params['Serwisowany w ASO'].fillna(params['Service record'])
    del params['Service record']
if 'Body type' in params:
    params['Typ'] = params['Typ'].fillna(params['Body type'])
    del params['Body type']
if 'Transmission' in params:
    params['Napęd'] = params['Napęd'].fillna(params['Transmission'])
    del params['Transmission']
if 'Mileage' in params:
    params['Przebieg'] = params['Przebieg'].fillna(params['Mileage'])
    del params['Mileage']
if 'Gearbox' in params:
    params['Skrzynia biegów'] = params['Skrzynia biegów'].fillna(params['Gearbox'])
    del params['Gearbox']
if 'Model' in params:
    params['Model pojazdu'] = params['Model pojazdu'].fillna(params['Model'])
    del params['Model']
if 'Engine capacity' in params:
    params['Pojemność skokowa'] = params['Pojemność skokowa'].fillna(params['Engine capacity'])
    del params['Engine capacity']
if 'Monthly payment value' in params:
    params['Miesięczna rata'] = params['Miesięczna rata'].fillna(params['Monthly payment value'])
    del params['Monthly payment value']
if 'Particle filter' in params:
    params['Filtr cząstek stałych'] = params['Filtr cząstek stałych'].fillna(params['Particle filter'])
    del params['Particle filter']
if 'Engine Code' in params:
    params['Kod Silnika'] = params['Kod Silnika'].fillna(params['Engine Code'])
    del params['Engine Code']
if 'Financing option' in params:
    params['Możliwość finansowania'] = params['Możliwość finansowania'].fillna(params['Financing option'])
    del params['Financing option']
if 'Acrylic' in params:
    params['Akryl (niemetalizowany)'] = params['Akryl (niemetalizowany)'].fillna(params['Acrylic'])
    del params['Acrylic']
if 'Fuel type' in params:
    params['Rodzaj paliwa'] = params['Rodzaj paliwa'].fillna(params['Fuel type'])
    del params['Fuel type']
if 'Metallic' in params:
    params['Metalik'] = params['Metalik'].fillna(params['Metallic'])
    del params['Metallic']
if 'Original owner' in params:
    params['Pierwszy właściciel'] = params['Pierwszy właściciel'].fillna(params['Original owner'])
    del params['Original owner']
if 'First registration' in params:
    params['Pierwsza rejestracja'] = params['Pierwsza rejestracja'].fillna(params['First registration'])
    del params['First registration']
if 'Color' in params:
    params['Kolor'] = params['Kolor'].fillna(params['Color'])
    del params['Color']
if 'Leasing concession' in params:
    params['Leasing'] = params['Leasing'].fillna(params['Leasing concession'])
    del params['Leasing concession']
if 'CO2 emissions' in params:
    params['Emisja CO2'] = params['Emisja CO2'].fillna(params['CO2 emissions'])
    del params['CO2 emissions']
if 'Nr of seats' in params:
    params['Liczba miejsc'] = params['Liczba miejsc'].fillna(params['Nr of seats'])
    del params['Nr of seats']
if 'No accident' in params:
    params['Bezwypadkowy'] = params['Bezwypadkowy'].fillna(params['No accident'])
    del params['No accident']
if 'Country of origin' in params:
    params['Kraj pochodzenia'] = params['Kraj pochodzenia'].fillna(params['Country of origin'])
    del params['Country of origin']
if 'Damaged' in params:
    params['Uszkodzony'] = params['Uszkodzony'].fillna(params['Damaged'])
    del params['Damaged']
if 'Registered in Poland' in params:
    params['Zarejestrowany w Polsce'] = params['Zarejestrowany w Polsce'].fillna(params['Registered in Poland'])
    del params['Registered in Poland']
if 'Version' in params:
    params['Wersja'] = params['Wersja'].fillna(params['Version'])
    del params['Version']    
if 'Pearl' in params:
    params['Perłowy'] = params['Perłowy'].fillna(params['Pearl'])
    del params['Pearl']
if 'Year' in params:
    params['Rok produkcji'] = params['Rok produkcji'].fillna(params['Year'])
    del params['Year']
if 'Engine power' in params:
    params['Moc'] = params['Moc'].fillna(params['Engine power'])
    del params['Engine power']
if 'Make' in params:
    params['Marka pojazdu'] = params['Marka pojazdu'].fillna(params['Make'])
    del params['Make']

In [130]:
params.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160258 entries, 7 to 160254
Data columns (total 45 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Bezwypadkowy                   160258 non-null  object 
 1   Liczba miejsc                  160258 non-null  int64  
 2   Kraj pochodzenia               160258 non-null  object 
 3   Pojemność skokowa              152799 non-null  float64
 4   Rodzaj paliwa                  160258 non-null  object 
 5   Skrzynia biegów                160258 non-null  object 
 6   Możliwość finansowania         160258 non-null  object 
 7   Pierwszy właściciel            160258 non-null  object 
 8   Oferta od                      160258 non-null  object 
 9   Liczba drzwi                   160258 non-null  int64  
 10  Metalik                        160258 non-null  object 
 11  Typ                            160258 non-null  object 
 12  Kolor                         

In [102]:
#fill empty fields
params = params.fillna(-1)

In [103]:
#Bezwypadkowy
params["Bezwypadkowy"].value_counts()

Tak    100708
-1      59550
Name: Bezwypadkowy, dtype: int64

In [104]:
#Liczba miejsc
params["Liczba miejsc"] = params["Liczba miejsc"].astype(int)
params["Liczba miejsc"].value_counts()

 5    123126
-1     17749
 7      8103
 4      7050
 2      1723
 9      1057
 6       567
 8       504
 3       375
 1         4
Name: Liczba miejsc, dtype: int64

In [105]:
#Kraj pochodzenia
params["Kraj pochodzenia"].value_counts()

-1                   51080
Niemcy               44882
Polska               39645
Francja               6023
Belgia                4887
Szwajcaria            2274
Holandia              2121
Stany Zjednoczone     2115
Włochy                1493
Austria               1259
Inny                  1228
Wielka Brytania        834
Dania                  815
Szwecja                679
Luksemburg             228
Kanada                 208
Czechy                 189
Hiszpania              120
Finlandia               33
Słowacja                28
Irlandia                28
Norwegia                17
Rosja                   14
Grecja                   9
Słowenia                 9
Litwa                    8
Monako                   7
Liechtenstein            6
Estonia                  5
Węgry                    5
Ukraina                  3
Rumunia                  2
Białoruś                 1
Bułgaria                 1
Chorwacja                1
Łotwa                    1
Name: Kraj pochodzenia, dtyp

In [106]:
#pojemnosc skokowa
params["Pojemność skokowa_jedn"] = params["Pojemność skokowa"].str[-3:]
params["Pojemność skokowa"] = params["Pojemność skokowa"].str[:-4]

In [107]:
params["Pojemność skokowa_jedn"].value_counts()

cm3    152799
Name: Pojemność skokowa_jedn, dtype: int64

In [108]:
#same unit. delete column Pojemność skokowa_jedn
if 'Pojemność skokowa_jedn' in params: del params['Pojemność skokowa_jedn']

In [109]:
params["Pojemność skokowa"] = params["Pojemność skokowa"].map(lambda x: str(x).replace(" ", "")).astype(float)


In [110]:
#Rodzaj paliwa
params["Rodzaj paliwa"].value_counts()

Diesel         81228
Benzyna        70663
Benzyna+LPG     7183
Hybryda          960
Elektryczny      120
Benzyna+CNG      102
Etanol             1
Wodór              1
Name: Rodzaj paliwa, dtype: int64

In [111]:
#Skrzynia biegów  
params["Skrzynia biegów"].value_counts()

Manualna                                 110510
Automatyczna hydrauliczna (klasyczna)     27974
Automatyczna dwusprzęgłowa (DCT, DSG)      9636
-1                                         8541
Automatyczna bezstopniowa (CVT)            2298
Półautomatyczna (ASG, Tiptronic)           1299
Name: Skrzynia biegów, dtype: int64

In [112]:
#Liczba drzwi
params["Liczba drzwi"] = params["Liczba drzwi"].astype(int)
params["Liczba drzwi"].value_counts()

 5     107640
 4      22885
-1      14626
 3      11017
 2       3909
 6        144
 7         18
 9          6
 8          6
 1          6
 30         1
Name: Liczba drzwi, dtype: int64

In [113]:
params[params["Liczba drzwi"]>20].head()

,Bezwypadkowy,Liczba miejsc,Kraj pochodzenia,Pojemność skokowa,Rodzaj paliwa,Skrzynia biegów,Możliwość finansowania,Pierwszy właściciel,Oferta od,Liczba drzwi,...,Liczba pozostałych rat,Opłata początkowa,Wartość wykupu,Akryl (niemetalizowany),Tuning,Kierownica po prawej (Anglik),Homologacja ciężarowa,Matowy,Zarejestrowany jako zabytek,VAT free
42070,-1,5,Inny,1600.0,Benzyna,Manualna,-1,-1,Firmy,30,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [114]:
df[df["id"]==42070].head()

,id,price_details,price_value,price_currency,features,seller_name,seller_type,seller_address,breadcrumb,offer_params
42070,42070,Cena Brutto,16500.0,PLN,"[ABS, Alarm, Elektryczne szyby przednie, Klima...","AUTO KOMIS KLT, SKUP*SPRZEDAŻ*ZAMIANA* SAMOCHO...",Dealer,"al. 29 Listopada 141 - 31-406 Kraków, Małopols...","[Osobowe, Suzuki, Swift, Suzuki Swift IV (2004...","{'Pojemność skokowa': '1 600 cm3', 'Model poja..."


In [115]:
#replace outlier: 30 with unknown value
params['Liczba drzwi'] = params['Liczba drzwi'].replace([30],-1)

In [123]:
#przebieg
params["Przebieg2"] = params["Przebieg"].astype(str).str[:-3]
params["Przebieg_jedn"] = params["Przebieg"].astype(str).str[-3:]
params.loc[params["Przebieg2"]=="",'Przebieg2'] = -1
params["Przebieg2"] = params["Przebieg2"].map(lambda x: str(x).replace(" ", "")).astype(int)

In [127]:
#delete column przebieg
if 'Przebieg' in params: del params['Przebieg']
if 'Przebieg_jedn' in params: del params['Przebieg_jedn']

In [122]:
#Moc
params["Moc2"] = params["Moc"].astype(str).str[:-3]
params["Moc_jedn"] = params["Moc"].astype(str).str[-3:]
params.loc[params["Moc2"]=="",'Moc2'] = -1
params["Moc2"] = params["Moc2"].map(lambda x: str(x).replace(" ", "")).astype(int)

In [128]:
#delete column Moc
if 'Moc' in params: del params['Moc']
if 'Moc_jedn' in params: del params['Moc_jedn']

In [129]:
params["Rok produkcji"] = params["Rok produkcji"].astype(int)

In [131]:

if 'Bezwypadkowy' not in df:
    df = pd.concat([df, params], axis=1)
df

,id,price_details,price_value,price_currency,features,seller_name,seller_type,seller_address,breadcrumb,offer_params,...,Wartość wykupu,Akryl (niemetalizowany),Tuning,Kierownica po prawej (Anglik),Homologacja ciężarowa,Matowy,Zarejestrowany jako zabytek,VAT free,Moc2,Przebieg2
7,7,"Cena Brutto, Faktura VAT",63900.0,PLN,"[ABS, Elektrycznie ustawiane lusterka, Poduszk...",TRANS-AUTO SP. Z O.O. DEALER MARKI ...,Dealer,"ŁUKASIEWICZA 1 - 19-300 Ełk, ełcki, Warmińsko-...","[Osobowe, Citroën, C4 Cactus]","{'Bezwypadkowy': 'Tak', 'Liczba miejsc': '5', ...",...,-1,-1,-1,-1,-1,-1,-1,-1,110,-1
10,10,Cena Brutto,41900.0,PLN,"[ABS, ASR (kontrola trakcji), Elektryczne szyb...",DECAR - RENAULT i DACIA,Dealer,"MODLIŃSKA 154 B - 03-170 Warszawa, Mazowieckie...","[Osobowe, Nissan, Juke]","{'Kolor': 'Czarny', 'Model pojazdu': 'Juke', '...",...,-1,-1,-1,-1,-1,-1,-1,-1,117,118
11,11,"Cena Brutto, Do negocjacji",17950.0,PLN,"[ABS, Elektryczne szyby przednie, Poduszka pow...",SKUP SPRZEDAŻ ZAMIANA RATY,Dealer,"Ruchocinek 63c - 62-230 Witkowo, gnieźnieński,...","[Osobowe, Opel, Astra, Opel Astra H (2004-2014)]","{'Bezwypadkowy': 'Tak', 'Liczba miejsc': '5', ...",...,-1,-1,-1,-1,-1,-1,-1,-1,90,106
14,14,"Cena Brutto, Do negocjacji",38000.0,PLN,"[ABS, Elektryczne szyby przednie, Poduszka pow...",None,Osoba prywatna,"Chróścice, opolski, Opolskie","[Osobowe, Audi, A4, Audi A4 B8 (2007-2015)]","{'Liczba miejsc': '5', 'Kraj pochodzenia': 'Ni...",...,-1,-1,-1,-1,-1,-1,-1,-1,160,172
16,16,Cena Brutto,21900.0,PLN,"[ABS, Elektryczne szyby przednie, Poduszka pow...",None,Osoba prywatna,"Trzeboszowice, nyski, Opolskie","[Osobowe, Ford, Fiesta, Ford Fiesta Mk7 (2008-)]","{'Bezwypadkowy': 'Tak', 'Liczba miejsc': '5', ...",...,-1,-1,-1,-1,-1,-1,-1,-1,82,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160245,160245,NaN,NaN,NaN,"[ABS, Elektryczne szyby przednie, Poduszka pow...",None,Osoba prywatna,"Leszno, Wielkopolskie","[Osobowe, Volkswagen, Passat, Volkswagen Passa...","{'Pojemność skokowa': '2 000 cm3', 'Model poja...",...,-1,-1,-1,-1,-1,-1,-1,-1,170,109
160246,160246,NaN,NaN,NaN,"[ABS, Elektryczne szyby przednie, Poduszka pow...",PTUH Adam Dudziak,Dealer,"Zieleniewo, ulica Szczecińska 59 - 78-100 Ziel...","[Osobowe, Citroën, C1, Citroën C1 II (2014-)]","{'Liczba miejsc': '4', 'Kraj pochodzenia': 'Da...",...,-1,-1,-1,-1,-1,-1,-1,-1,69,40
160247,160247,NaN,NaN,NaN,"[ABS, Elektryczne szyby przednie, Poduszka pow...",R-CAR RAFAŁ ŻYWIŃSKI,Dealer,"ul.Krakowska 259A - 43-300 Bielsko-Biała, Śląs...","[Osobowe, Opel, Meriva, Opel Meriva I (2002-20...","{'Bezwypadkowy': 'Tak', 'Liczba miejsc': '5', ...",...,-1,-1,-1,-1,-1,-1,-1,-1,105,132
160251,160251,NaN,NaN,NaN,"[ABS, Elektryczne szyby przednie, Poduszka pow...",None,Osoba prywatna,"Toruń, Kujawsko-pomorskie","[Osobowe, Peugeot, 207]","{'Liczba miejsc': '5', 'Model pojazdu': '207',...",...,-1,-1,-1,-1,-1,-1,-1,-1,92,76


In [132]:
#breadcrumbs
df3 = pd.DataFrame(df['breadcrumb'].to_list(), columns=['rodzaj','marka','k1', 'k2', 'k3'])
breadcrumb_df = df3[['rodzaj','marka']]

top20 = breadcrumb_df.marka.value_counts().nlargest(20).index
breadcrumb_df["marka_top20"] = breadcrumb_df["marka"].map(lambda x: x in top20)
df = pd.concat([df, breadcrumb_df], axis=1, join="inner")

<ipython-input-132-356d90aee23b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  breadcrumb_df["marka_top20"] = breadcrumb_df["marka"].map(lambda x: x in top20)


In [133]:
#features
def to_1D(series):
 return pd.Series([x for _list in series for x in _list])

def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: item in x)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)

In [134]:
unique_features = list(set(to_1D(df["features"]).values))
features_bool = boolean_df(df["features"], unique_features).astype(int)
df = pd.concat([df, features_bool], axis=1)
df['features_count'] = df["features"].str.len()

In [135]:
#grouping features
if "parkowanie" not in df:
    df["parkowanie"] = df["Kamera cofania"] + df["Czujniki parkowania przednie"]+ df["Czujniki parkowania tylne"] + df["Czujnik martwego pola"] + df["Asystent parkowania"]
    df["klimatyzacja"] = df["Klimatyzacja manualna"] + 2* df["Klimatyzacja automatyczna"]+ 2* df["Klimatyzacja czterostrefowa"]+ df["Klimatyzacja dwustrefowa"]
    df["podgrzewanie"] = df["Podgrzewana przednia szyba"] + df["Podgrzewane przednie siedzenia"] + df["Podgrzewane tylne siedzenia"]+ df["Ogrzewanie postojowe"]
    df["poduszki"] = df["Poduszki boczne przednie"]+ df["Poduszka powietrzna chroniąca kolana"]+ df["Poduszki boczne tylne"] +df["Poduszka powietrzna kierowcy"] + df["Poduszka powietrzna pasażera"]
    df["wspomaganie"] = df["ASR (kontrola trakcji)"] + df["Wspomaganie kierownicy"] + df["ESP (stabilizacja toru jazdy)"] + df["ABS"] + df["Asystent pasa ruchu"]
    df["tapicerka"] = df["Tapicerka welurowa"] +2* df["Tapicerka skórzana"]
    df["czujniki"]  = df["Czujnik deszczu"] + df["Czujnik zmierzchu"]
    df["tempomat"] = df["Tempomat"] + 2* df["Ogranicznik prędkości"] + 3* df["Tempomat aktywny"]

In [136]:
df.columns

Index(['id', 'price_details', 'price_value', 'price_currency', 'features',
       'seller_name', 'seller_type', 'seller_address', 'breadcrumb',
       'offer_params',
       ...
       'Alufelgi', 'features_count', 'parkowanie', 'klimatyzacja',
       'podgrzewanie', 'poduszki', 'wspomaganie', 'tapicerka', 'czujniki',
       'tempomat'],
      dtype='object', length=137)

In [137]:
# outlier 106447
df[ df["price_value"] > 100000000 ]


,id,price_details,price_value,price_currency,features,seller_name,seller_type,seller_address,breadcrumb,offer_params,...,Alufelgi,features_count,parkowanie,klimatyzacja,podgrzewanie,poduszki,wspomaganie,tapicerka,czujniki,tempomat
106447,106447,Cena Brutto,1.000000e+13,PLN,"[ABS, Elektrycznie ustawiane lusterka, Poduszk...",None,Osoba prywatna,"Lubań, lubański, Dolnośląskie","[Osobowe, Honda, Jazz, Honda Jazz IV (2015-)]","{'Pojemność skokowa': '9 999 cm3', 'Model poja...",...,0,16,1,1,0,2,4,0,0,0


In [138]:
#categorical features
obj_feats = df.select_dtypes(object).columns

for feat in obj_feats:
    some_value = df[feat].values[0]
    if isinstance(some_value, list): continue
    if isinstance(some_value, dict): continue
    if "price" in feat: continue

    df["{}_cat".format(feat)] = df[feat].factorize()[0]
    
cat_feats = [x for x in df.columns if "_cat" in x]
cat_feats


['seller_name_cat',
 'seller_type_cat',
 'seller_address_cat',
 'Bezwypadkowy_cat',
 'Kraj pochodzenia_cat',
 'Rodzaj paliwa_cat',
 'Skrzynia biegów_cat',
 'Możliwość finansowania_cat',
 'Pierwszy właściciel_cat',
 'Oferta od_cat',
 'Metalik_cat',
 'Typ_cat',
 'Kolor_cat',
 'Model pojazdu_cat',
 'Stan_cat',
 'Kategoria_cat',
 'Serwisowany w ASO_cat',
 'Napęd_cat',
 'Leasing_cat',
 'Marka pojazdu_cat',
 'Faktura VAT_cat',
 'Pierwsza rejestracja_cat',
 'Zarejestrowany w Polsce_cat',
 'VAT marża_cat',
 'Wersja_cat',
 'VIN_cat',
 'Perłowy_cat',
 'Uszkodzony_cat',
 'Kod Silnika_cat',
 'Filtr cząstek stałych_cat',
 'Emisja CO2_cat',
 'Miesięczna rata_cat',
 'Liczba pozostałych rat_cat',
 'Opłata początkowa_cat',
 'Wartość wykupu_cat',
 'Akryl (niemetalizowany)_cat',
 'Tuning_cat',
 'Kierownica po prawej (Anglik)_cat',
 'Homologacja ciężarowa_cat',
 'Matowy_cat',
 'Zarejestrowany jako zabytek_cat',
 'VAT free_cat',
 'rodzaj_cat',
 'marka_cat']

# Models

In [139]:
df_train = df[ ~df["price_value"].isnull() ].copy()
df_test = df[ df["price_value"].isnull() ].copy()

df_train.shape, df_test.shape

((80129, 181), (80129, 181))

In [140]:
def get_feats(df, black_list=['id', 'price_currency', 'price_value', 'VIN_cat', 'Cena Brutto', 'Cena Netto', 'Do negocjacji']):
    num_feats = df.select_dtypes(np.number).columns
    return [x for x in num_feats if x not in black_list]

In [141]:
feats = get_feats(df_train)
feats

['Liczba miejsc',
 'Pojemność skokowa',
 'Liczba drzwi',
 'Rok produkcji',
 'Moc2',
 'Przebieg2',
 'Przyciemniane szyby',
 'Wielofunkcyjna kierownica',
 'Zmieniarka CD',
 'Odtwarzacz DVD',
 'Tempomat aktywny',
 'ASR (kontrola trakcji)',
 'Elektryczne szyby tylne',
 'Podgrzewane tylne siedzenia',
 'Czujnik deszczu',
 'ESP (stabilizacja toru jazdy)',
 'Światła LED',
 'Poduszka powietrzna kierowcy',
 'Poduszka powietrzna chroniąca kolana',
 'Łopatki zmiany biegów',
 'Poduszki boczne tylne',
 'Tapicerka skórzana',
 'Kurtyny powietrzne',
 'Podgrzewana przednia szyba',
 'Kamera cofania',
 'Klimatyzacja dwustrefowa',
 'Alarm',
 'Gniazdo USB',
 'ABS',
 'Isofix',
 'Klimatyzacja manualna',
 'Podgrzewane lusterka boczne',
 'Klimatyzacja automatyczna',
 'MP3',
 'Poduszki boczne przednie',
 'Elektrochromatyczne lusterko wsteczne',
 'Światła przeciwmgielne',
 'System Start-Stop',
 'Światła Xenonowe',
 'Czujnik zmierzchu',
 'Immobilizer',
 'Czujniki parkowania przednie',
 'Radio fabryczne',
 'HUD (wy

In [142]:
X_train = df_train[feats]
y_train = df_train["price_value"]

X_test = df_test[feats]

In [143]:
def check_model(df, feats, model, cv=5, scoring="neg_mean_absolute_error", show_eli5=True, show_over=True):
    df_train = df[ ~df["price_value"].isnull() ].copy()
    df_test = df[ df["price_value"].isnull() ].copy()

    X_train = df_train[feats]
    y_train = df_train["price_value"]

    X_test = df_test[feats]
    
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring=scoring)
    result = np.mean(scores), np.std(scores)
    
    if show_eli5:
        model.fit(X_train, y_train)
        print(result)
        return eli5.show_weights(model, feature_names=feats)
    if show_over:
        return plot_learning_curve(model, X_train, y_train, cv);
    return result

In [144]:
def check_log_model(df, feats, model, cv=5, scoring=mean_absolute_error, show_eli5=True):
    df_train = df[ ~df["price_value"].isnull() ].copy()

    X = df_train[feats]
    y = df_train["price_value"]
    y_log = np.log(y)
    
    cv = KFold(n_splits=cv, shuffle=True, random_state=0)
    scores = []
    for train_idx, test_idx in cv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_log_train, y_test = y_log.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_log_train)
        y_log_pred = model.predict(X_test)
        y_pred = np.exp(y_log_pred)

        score = scoring(y_test, y_pred)
        scores.append(score)
        
    result = np.mean(scores), np.std(scores)
    
    if show_eli5:
        model.fit(X, y_log)
        print(result)
        return eli5.show_weights(model, feature_names=feats)

    return result

In [152]:
check_log_model(df, feats, ctb(max_depth=7, n_estimators=50, learning_rate=0.3, random_state=0))

0:	learn: 0.8055528	total: 21ms	remaining: 1.03s
1:	learn: 0.6474281	total: 44ms	remaining: 1.06s
2:	learn: 0.5416425	total: 69.6ms	remaining: 1.09s
3:	learn: 0.4757801	total: 92.8ms	remaining: 1.07s
4:	learn: 0.4323385	total: 115ms	remaining: 1.04s
5:	learn: 0.4059878	total: 134ms	remaining: 982ms
6:	learn: 0.3867521	total: 157ms	remaining: 965ms
7:	learn: 0.3745889	total: 177ms	remaining: 928ms
8:	learn: 0.3648873	total: 197ms	remaining: 898ms
9:	learn: 0.3565790	total: 215ms	remaining: 861ms
10:	learn: 0.3497180	total: 236ms	remaining: 838ms
11:	learn: 0.3429590	total: 261ms	remaining: 826ms
12:	learn: 0.3387612	total: 282ms	remaining: 803ms
13:	learn: 0.3334521	total: 304ms	remaining: 782ms
14:	learn: 0.3301259	total: 321ms	remaining: 748ms
15:	learn: 0.3256457	total: 339ms	remaining: 721ms
16:	learn: 0.3225684	total: 361ms	remaining: 701ms
17:	learn: 0.3202080	total: 379ms	remaining: 674ms
18:	learn: 0.3179962	total: 397ms	remaining: 648ms
19:	learn: 0.3159477	total: 415ms	remaini

Weight,Feature
0.4215,Rok produkcji
0.1327,Moc2
0.0923,Przebieg2
0.0590,Marka pojazdu_cat
0.0516,Napęd_cat
0.0352,Uszkodzony_cat
0.0328,Typ_cat
0.0328,Pojemność skokowa
0.0172,Skrzynia biegów_cat
0.0164,features_count


# Submit

In [65]:
df_test["price_value"] = model_xgb.predict(X_test)

df_test[["id", "price_value"]]

df_test[["id", "price_value"]].to_csv("../output/xgb3.csv", index=False)